In [1]:
import pandas as pd
import numpy as np
import json
import os
import unidecode
import unicodedata as ud
import regex as re
import xml.etree.ElementTree

In [42]:
MY_COLS = [
    'date', 'title', 'industry', 'mapped_industry', 'level', 'min_salary', 'max_salary', 'min_year', 'max_year', 'city', 'description', 'requirements'
]

In [7]:
def split_attributes(x):
    return x[1:-1].replace("'","")

In [13]:
def modified_split(x):
    return x.replace(", ", ",")

In [11]:
def get_mapping_list(series, mapped_df, mapped_col, split_func=split_attributes):
    ct_list = []
    if split_func is not None:
        modified_series = series.apply(split_func)
    else: modified_series = series
    for ct_l in modified_series:
        temp_ls = []
        for ct in ct_l.split(','):
            temp_ls.append(mapped_df[mapped_col][ct.strip()])
        ct_list.append(', '.join(str(t) for t in temp_ls))
    return ct_list

In [3]:
vnw_r = pd.read_csv('data/csv/vietnamworks.csv')

In [4]:
vnw_r

,comp_name,tittle_job,local,date,level,industry,skills,description,requirement,welfare,min_salary,max_salary,min_year,max_year
0,Navigos Group,Senior Recruitment Consultant (Industrial Manu...,['Hồ Chí Minh'],2023-04-19,Nhân viên,"[5, 34, 52]","Recruitment, Business Development, Talent Acqu...","Navigos Search, a part of Navigos Group, is th...",- College Degree and above - At least 3 year...,Attractive commission scheme; 26 paid leave; P...,NaN,NaN,3.0,NaN
1,Công Ty TNHH Hyosung Financial System VINA,Nhân Viên Chất Lượng IQC ( Ưu Tiên Biết Tiếng ...,"['Bắc Ninh', 'Hà Nội']",2023-04-18,Nhân viên,"[39, 42, 60]","Quản Lý Chất Lượng IQC, IQC, CAD, Quản Lí Chất...",- Phụ trách quản lý chất lượng hàng kim loại. ...,- Có kinh nghiệm từ 2 năm trở lên làm chất lượ...,"Chế độ lương, thưởng, phúc lợi cạnh tranh; Có ...",NaN,NaN,2.0,NaN
2,Công Ty Cổ Phần Dịch Vụ Và Địa Ốc Đất Xanh Miề...,Chuyên Viên Tư Vấn Bất Động Sản - Lương Cứng L...,['Hà Nội'],2023-04-21,Nhân viên,"[5, 7, 52]","Kinh Doanh, Bất Động Sản, Kinh Doanh Bất Động ...",Đất Xanh Miền Bắc là công ty bất động sản hàng...,"- Đam mê kinh doanh, muốn kiếm nhiều tiền và m...",Lương cứng từ 6tr-10tr có cả ngay khi thử việc...,23460000.0,NaN,NaN,NaN
3,Ngân Hàng TNHH Indovina – Hội Sở,Banking Product Development Officer,['Hồ Chí Minh'],2023-04-29,Nhân viên,"[30, 33, 50]","Banking Products, Corporate Banking, Retail Ba...",Tasks: - Analyze competition of Banking produc...,Job requirements: - Male/ Female with Bachelor...,13th month salary; Health insurance and Social...,NaN,NaN,2.0,NaN
4,Pan Services Hà Nội,Giám Sát Dịch Vụ Vệ Sinh Tòa Nhà Làm Việc Tại ...,['Hà Nội'],2023-04-21,Nhân viên,"[8, 25, 31]","Giám Sát, Quản Lý Dịch Vụ, Giám Sát Điều Hành,...",I. Thông tin chung: - Chức danh: ...,"– Nữ từ 24 – 40 tuổi, có sức khỏe tốt – Tốt ng...","Hoa hồng, lương tháng 13,...; Chăm sóc sức khỏ...",8680200.0,11260800.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9550,Page Group Vietnam,"General Manager, Financial Services","['Hồ Chí Minh', 'Hà Nội']",2023-01-04,Giám Đốc và Cấp Cao Hơn,"[10, 33, 50]","Management, Sales Management Coaching, IT Busi...",We are seeking an experienced and motivated Ge...,Minimum of 20 years of experience in sales and...,"Attractive compensation package, including a c...",NaN,NaN,NaN,NaN
9551,Asiafoods Corporation,Giám Đốc Nhân Sự,['Hồ Chí Minh'],2023-01-04,Giám Đốc và Cấp Cao Hơn,"[10, 18, 34]","Tuyển Dụng, Quản Trị Nhân Sự, Quản Trị Nhân Lự...","1/ Cấu trúc tổ chức - Xây dựng, thiết kế cấu ...",- Cử nhân/ Thạc sĩ trở lên trong các ngành: QT...,"Lương tháng 13, AOP; Được tham gia các khóa đà...",82110000.0,140760000.0,5.0,NaN
9552,Navigos Search's Client,HSE Supervisor Upto 25 Millions,['Hà Nội'],2023-01-04,Trưởng phòng,"[2, 31]","HSE, SHE, An Toàn Lao Động, Phòng Cháy Chữa Cháy",Khách hàng của Navigos Search là công ty sản x...,- Nam từ 28 - 38 tuổi - Tốt nghiệp các trường ...,"Chế độ đãi ngộ hấp dẫn cạnh tranh, xứng đáng t...",16422000.0,25806000.0,2.0,NaN
9553,Navigos Search's Client,Rất Gấp Kỹ Sư Cải Tiến,['Hà Nội'],2023-01-04,Nhân viên,"[41, 42]","Kaizen, Innovation Ideas, Cải Tiến Tự Động Hóa...",Khách hàng của Navigos Search là công ty R&D n...,"- Nam/nữ, dưới 27 tuổi - Tốt nghiệp Đại học tr...","Chế độ đãi ngộ hấp dẫn cạnh tranh, xứng đáng t...",11730000.0,18768000.0,3.0,NaN


In [5]:
vnw_re = vnw_r[['date', 'tittle_job', 'industry', 'level', 'min_year', 'max_year', 'min_salary', 'max_salary', 'local', 'description', 'requirement']].copy()
vnw_re.columns = ['date', 'title', 'industry', 'level', 'min_year', 'max_year', 'min_salary', 'max_salary', 'city', 'description', 'requirement']
vnw_re

,date,title,industry,level,min_year,max_year,min_salary,max_salary,city,description,requirement
0,2023-04-19,Senior Recruitment Consultant (Industrial Manu...,"[5, 34, 52]",Nhân viên,3.0,NaN,NaN,NaN,['Hồ Chí Minh'],"Navigos Search, a part of Navigos Group, is th...",- College Degree and above - At least 3 year...
1,2023-04-18,Nhân Viên Chất Lượng IQC ( Ưu Tiên Biết Tiếng ...,"[39, 42, 60]",Nhân viên,2.0,NaN,NaN,NaN,"['Bắc Ninh', 'Hà Nội']",- Phụ trách quản lý chất lượng hàng kim loại. ...,- Có kinh nghiệm từ 2 năm trở lên làm chất lượ...
2,2023-04-21,Chuyên Viên Tư Vấn Bất Động Sản - Lương Cứng L...,"[5, 7, 52]",Nhân viên,NaN,NaN,23460000.0,NaN,['Hà Nội'],Đất Xanh Miền Bắc là công ty bất động sản hàng...,"- Đam mê kinh doanh, muốn kiếm nhiều tiền và m..."
3,2023-04-29,Banking Product Development Officer,"[30, 33, 50]",Nhân viên,2.0,NaN,NaN,NaN,['Hồ Chí Minh'],Tasks: - Analyze competition of Banking produc...,Job requirements: - Male/ Female with Bachelor...
4,2023-04-21,Giám Sát Dịch Vụ Vệ Sinh Tòa Nhà Làm Việc Tại ...,"[8, 25, 31]",Nhân viên,1.0,NaN,8680200.0,11260800.0,['Hà Nội'],I. Thông tin chung: - Chức danh: ...,"– Nữ từ 24 – 40 tuổi, có sức khỏe tốt – Tốt ng..."
...,...,...,...,...,...,...,...,...,...,...,...
9550,2023-01-04,"General Manager, Financial Services","[10, 33, 50]",Giám Đốc và Cấp Cao Hơn,NaN,NaN,NaN,NaN,"['Hồ Chí Minh', 'Hà Nội']",We are seeking an experienced and motivated Ge...,Minimum of 20 years of experience in sales and...
9551,2023-01-04,Giám Đốc Nhân Sự,"[10, 18, 34]",Giám Đốc và Cấp Cao Hơn,5.0,NaN,82110000.0,140760000.0,['Hồ Chí Minh'],"1/ Cấu trúc tổ chức - Xây dựng, thiết kế cấu ...",- Cử nhân/ Thạc sĩ trở lên trong các ngành: QT...
9552,2023-01-04,HSE Supervisor Upto 25 Millions,"[2, 31]",Trưởng phòng,2.0,NaN,16422000.0,25806000.0,['Hà Nội'],Khách hàng của Navigos Search là công ty sản x...,- Nam từ 28 - 38 tuổi - Tốt nghiệp các trường ...
9553,2023-01-04,Rất Gấp Kỹ Sư Cải Tiến,"[41, 42]",Nhân viên,3.0,NaN,11730000.0,18768000.0,['Hà Nội'],Khách hàng của Navigos Search là công ty R&D n...,"- Nam/nữ, dưới 27 tuổi - Tốt nghiệp Đại học tr..."


### Mapping text industry for vietnamworks

In [8]:
vnw_re['industry'] = vnw_re['industry'].apply(split_attributes)

In [18]:
industries = pd.read_csv("data/topcv/csv/mapping-titles - industries.csv")
industries['id'] = industries['id'].apply(str)
industries.set_index('id', inplace=True)

In [20]:
vnw_re['mapped_industry'] = get_mapping_list(series=vnw_re['industry'], mapped_df=industries, mapped_col='name', split_func=modified_split)

In [21]:
vnw_re

,date,title,industry,level,min_year,max_year,min_salary,max_salary,city,description,requirement,mapped_industry
0,2023-04-19,Senior Recruitment Consultant (Industrial Manu...,"5, 34, 52",Nhân viên,3.0,NaN,NaN,NaN,['Hồ Chí Minh'],"Navigos Search, a part of Navigos Group, is th...",- College Degree and above - At least 3 year...,"Bán buôn - Bán lẻ - Quản lý cửa hàng, Nhân sự,..."
1,2023-04-18,Nhân Viên Chất Lượng IQC ( Ưu Tiên Biết Tiếng ...,"39, 42, 60",Nhân viên,2.0,NaN,NaN,NaN,"['Bắc Ninh', 'Hà Nội']",- Phụ trách quản lý chất lượng hàng kim loại. ...,- Có kinh nghiệm từ 2 năm trở lên làm chất lượ...,"Quản lý tiêu chuẩn và chất lượng, Sản xuất - L..."
2,2023-04-21,Chuyên Viên Tư Vấn Bất Động Sản - Lương Cứng L...,"5, 7, 52",Nhân viên,NaN,NaN,23460000.0,NaN,['Hà Nội'],Đất Xanh Miền Bắc là công ty bất động sản hàng...,"- Đam mê kinh doanh, muốn kiếm nhiều tiền và m...","Bán buôn - Bán lẻ - Quản lý cửa hàng, Bất động..."
3,2023-04-29,Banking Product Development Officer,"30, 33, 50",Nhân viên,2.0,NaN,NaN,NaN,['Hồ Chí Minh'],Tasks: - Analyze competition of Banking produc...,Job requirements: - Male/ Female with Bachelor...,"Marketing, Ngân hàng, Tài chính - Đầu tư"
4,2023-04-21,Giám Sát Dịch Vụ Vệ Sinh Tòa Nhà Làm Việc Tại ...,"8, 25, 31",Nhân viên,1.0,NaN,8680200.0,11260800.0,['Hà Nội'],I. Thông tin chung: - Chức danh: ...,"– Nữ từ 24 – 40 tuổi, có sức khỏe tốt – Tốt ng...","Chăm sóc khách hàng, Khách sạn - Nhà hàng - Du..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9550,2023-01-04,"General Manager, Financial Services","10, 33, 50",Giám Đốc và Cấp Cao Hơn,NaN,NaN,NaN,NaN,"['Hồ Chí Minh', 'Hà Nội']",We are seeking an experienced and motivated Ge...,Minimum of 20 years of experience in sales and...,"Quản lý điều hành, Ngân hàng, Tài chính - Đầu tư"
9551,2023-01-04,Giám Đốc Nhân Sự,"10, 18, 34",Giám Đốc và Cấp Cao Hơn,5.0,NaN,82110000.0,140760000.0,['Hồ Chí Minh'],"1/ Cấu trúc tổ chức - Xây dựng, thiết kế cấu ...",- Cử nhân/ Thạc sĩ trở lên trong các ngành: QT...,"Quản lý điều hành, Hành chính - Thư ký, Nhân sự"
9552,2023-01-04,HSE Supervisor Upto 25 Millions,"2, 31",Trưởng phòng,2.0,NaN,16422000.0,25806000.0,['Hà Nội'],Khách hàng của Navigos Search là công ty sản x...,- Nam từ 28 - 38 tuổi - Tốt nghiệp các trường ...,"An toàn lao động, Môi trường - Xử lý chất thải"
9553,2023-01-04,Rất Gấp Kỹ Sư Cải Tiến,"41, 42",Nhân viên,3.0,NaN,11730000.0,18768000.0,['Hà Nội'],Khách hàng của Navigos Search là công ty R&D n...,"- Nam/nữ, dưới 27 tuổi - Tốt nghiệp Đại học tr...","Sản phẩm công nghiệp, Sản xuất - Lắp ráp - Chế..."


### Mapping levels for vietnamworks

In [23]:
levels = pd.read_csv("data/topcv/csv/mapping-levels - levels.csv")
levels.columns = ['mapped_id', 'name']
vnw_levels = pd.read_csv("data/topcv/csv/mapping-levels - vnw-levels.csv")

In [24]:
vnw_levels

,id,name,mapped_id
0,1,Mới Tốt Nghiệp,8
1,3,Giám Đốc và Cấp Cao Hơn,2
2,5,Nhân viên,6
3,7,Trưởng phòng,3
4,8,Thực tập sinh/Sinh viên,9


In [25]:
mapped_levels = vnw_levels.merge(levels, on='mapped_id', how='left')[['name_x', 'name_y']].set_index('name_x')

In [26]:
vnw_re['mapped_level'] = get_mapping_list(series=vnw_re['level'], mapped_df=mapped_levels, mapped_col='name_y', split_func=None)

In [27]:
vnw_re

,date,title,industry,level,min_year,max_year,min_salary,max_salary,city,description,requirement,mapped_industry,mapped_level
0,2023-04-19,Senior Recruitment Consultant (Industrial Manu...,"5, 34, 52",Nhân viên,3.0,NaN,NaN,NaN,['Hồ Chí Minh'],"Navigos Search, a part of Navigos Group, is th...",- College Degree and above - At least 3 year...,"Bán buôn - Bán lẻ - Quản lý cửa hàng, Nhân sự,...",Chuyên viên - Nhân viên
1,2023-04-18,Nhân Viên Chất Lượng IQC ( Ưu Tiên Biết Tiếng ...,"39, 42, 60",Nhân viên,2.0,NaN,NaN,NaN,"['Bắc Ninh', 'Hà Nội']",- Phụ trách quản lý chất lượng hàng kim loại. ...,- Có kinh nghiệm từ 2 năm trở lên làm chất lượ...,"Quản lý tiêu chuẩn và chất lượng, Sản xuất - L...",Chuyên viên - Nhân viên
2,2023-04-21,Chuyên Viên Tư Vấn Bất Động Sản - Lương Cứng L...,"5, 7, 52",Nhân viên,NaN,NaN,23460000.0,NaN,['Hà Nội'],Đất Xanh Miền Bắc là công ty bất động sản hàng...,"- Đam mê kinh doanh, muốn kiếm nhiều tiền và m...","Bán buôn - Bán lẻ - Quản lý cửa hàng, Bất động...",Chuyên viên - Nhân viên
3,2023-04-29,Banking Product Development Officer,"30, 33, 50",Nhân viên,2.0,NaN,NaN,NaN,['Hồ Chí Minh'],Tasks: - Analyze competition of Banking produc...,Job requirements: - Male/ Female with Bachelor...,"Marketing, Ngân hàng, Tài chính - Đầu tư",Chuyên viên - Nhân viên
4,2023-04-21,Giám Sát Dịch Vụ Vệ Sinh Tòa Nhà Làm Việc Tại ...,"8, 25, 31",Nhân viên,1.0,NaN,8680200.0,11260800.0,['Hà Nội'],I. Thông tin chung: - Chức danh: ...,"– Nữ từ 24 – 40 tuổi, có sức khỏe tốt – Tốt ng...","Chăm sóc khách hàng, Khách sạn - Nhà hàng - Du...",Chuyên viên - Nhân viên
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9550,2023-01-04,"General Manager, Financial Services","10, 33, 50",Giám Đốc và Cấp Cao Hơn,NaN,NaN,NaN,NaN,"['Hồ Chí Minh', 'Hà Nội']",We are seeking an experienced and motivated Ge...,Minimum of 20 years of experience in sales and...,"Quản lý điều hành, Ngân hàng, Tài chính - Đầu tư",Quản lý cấp cao
9551,2023-01-04,Giám Đốc Nhân Sự,"10, 18, 34",Giám Đốc và Cấp Cao Hơn,5.0,NaN,82110000.0,140760000.0,['Hồ Chí Minh'],"1/ Cấu trúc tổ chức - Xây dựng, thiết kế cấu ...",- Cử nhân/ Thạc sĩ trở lên trong các ngành: QT...,"Quản lý điều hành, Hành chính - Thư ký, Nhân sự",Quản lý cấp cao
9552,2023-01-04,HSE Supervisor Upto 25 Millions,"2, 31",Trưởng phòng,2.0,NaN,16422000.0,25806000.0,['Hà Nội'],Khách hàng của Navigos Search là công ty sản x...,- Nam từ 28 - 38 tuổi - Tốt nghiệp các trường ...,"An toàn lao động, Môi trường - Xử lý chất thải",Quản lý cấp trung
9553,2023-01-04,Rất Gấp Kỹ Sư Cải Tiến,"41, 42",Nhân viên,3.0,NaN,11730000.0,18768000.0,['Hà Nội'],Khách hàng của Navigos Search là công ty R&D n...,"- Nam/nữ, dưới 27 tuổi - Tốt nghiệp Đại học tr...","Sản phẩm công nghiệp, Sản xuất - Lắp ráp - Chế...",Chuyên viên - Nhân viên


### Mapping cities for vietnamworks

In [28]:
cities = pd.read_csv("data/topcv/csv/mapping-cities - cities.csv")
cities.columns = ['mapped_id', 'name', 'nameEn']
vnw_cities = pd.read_csv("data/topcv/csv/mapping-cities - vnw-cities.csv")

In [30]:
mapped_cities = vnw_cities.merge(cities, on='mapped_id', how='left')[['name_x', 'name_y']].set_index('name_x')

In [31]:
mapped_cities.index

Index(['An Giang', 'Bà Rịa - Vũng Tàu', 'Bắc Giang', 'Bắc Kạn', 'Bạc Liêu',
       'Bắc Ninh', 'Bến Tre', 'Bình Định', 'Bình Dương', 'Bình Phước',
       'Bình Thuận', 'Cà Mau', 'Cần Thơ', 'Cao Bằng', 'Đà Nẵng', 'Đắk Lắk',
       'Đắk Nông', 'ĐBSCL', 'Điện Biên', 'Đồng Nai', 'Đồng Tháp', 'Gia Lai',
       'Hà Giang', 'Hà Nam', 'Hà Nội', 'Hà Tĩnh', 'Hải Dương', 'Hải Phòng',
       'Hậu Giang', 'Hòa Bình', 'Hưng Yên', 'Khác', 'Khánh Hòa', 'Kiên Giang',
       'Kon Tum', 'Lai Châu', 'Lâm Đồng', 'Lạng Sơn', 'Lào Cai', 'Long An',
       'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận', 'Phú Thọ', 'Phú Yên',
       'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi', 'Quảng Ninh', 'Quảng Trị',
       'Quốc tế', 'Sóc Trăng', 'Sơn La', 'Tây Ninh', 'Thái Bình',
       'Thái Nguyên', 'Thanh Hóa', 'Thừa Thiên Huế', 'Tiền Giang',
       'Hồ Chí Minh', 'Trà Vinh', 'Tuyên Quang', 'Vĩnh Long', 'Vĩnh Phúc',
       'Yên Bái'],
      dtype='object', name='name_x')

In [32]:
vnw_re['mapped_city'] = get_mapping_list(vnw_re['city'], mapped_cities, 'name_y')

In [43]:
vnw_fi = vnw_re[
    ['date',
 'title',
 'industry',
 'mapped_industry',
 'mapped_level',
 'min_salary',
 'max_salary',
 'min_year',
 'max_year',
 'mapped_city',
 'description',
 'requirement']
].copy()
vnw_fi.columns = MY_COLS

In [44]:
vnw_fi

,date,title,industry,mapped_industry,level,min_salary,max_salary,min_year,max_year,city,description,requirements
0,2023-04-19,Senior Recruitment Consultant (Industrial Manu...,"5, 34, 52","Bán buôn - Bán lẻ - Quản lý cửa hàng, Nhân sự,...",Chuyên viên - Nhân viên,NaN,NaN,3.0,NaN,TP.HCM,"Navigos Search, a part of Navigos Group, is th...",- College Degree and above - At least 3 year...
1,2023-04-18,Nhân Viên Chất Lượng IQC ( Ưu Tiên Biết Tiếng ...,"39, 42, 60","Quản lý tiêu chuẩn và chất lượng, Sản xuất - L...",Chuyên viên - Nhân viên,NaN,NaN,2.0,NaN,"Bắc Ninh, Hà Nội",- Phụ trách quản lý chất lượng hàng kim loại. ...,- Có kinh nghiệm từ 2 năm trở lên làm chất lượ...
2,2023-04-21,Chuyên Viên Tư Vấn Bất Động Sản - Lương Cứng L...,"5, 7, 52","Bán buôn - Bán lẻ - Quản lý cửa hàng, Bất động...",Chuyên viên - Nhân viên,23460000.0,NaN,NaN,NaN,Hà Nội,Đất Xanh Miền Bắc là công ty bất động sản hàng...,"- Đam mê kinh doanh, muốn kiếm nhiều tiền và m..."
3,2023-04-29,Banking Product Development Officer,"30, 33, 50","Marketing, Ngân hàng, Tài chính - Đầu tư",Chuyên viên - Nhân viên,NaN,NaN,2.0,NaN,TP.HCM,Tasks: - Analyze competition of Banking produc...,Job requirements: - Male/ Female with Bachelor...
4,2023-04-21,Giám Sát Dịch Vụ Vệ Sinh Tòa Nhà Làm Việc Tại ...,"8, 25, 31","Chăm sóc khách hàng, Khách sạn - Nhà hàng - Du...",Chuyên viên - Nhân viên,8680200.0,11260800.0,1.0,NaN,Hà Nội,I. Thông tin chung: - Chức danh: ...,"– Nữ từ 24 – 40 tuổi, có sức khỏe tốt – Tốt ng..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9550,2023-01-04,"General Manager, Financial Services","10, 33, 50","Quản lý điều hành, Ngân hàng, Tài chính - Đầu tư",Quản lý cấp cao,NaN,NaN,NaN,NaN,"TP.HCM, Hà Nội",We are seeking an experienced and motivated Ge...,Minimum of 20 years of experience in sales and...
9551,2023-01-04,Giám Đốc Nhân Sự,"10, 18, 34","Quản lý điều hành, Hành chính - Thư ký, Nhân sự",Quản lý cấp cao,82110000.0,140760000.0,5.0,NaN,TP.HCM,"1/ Cấu trúc tổ chức - Xây dựng, thiết kế cấu ...",- Cử nhân/ Thạc sĩ trở lên trong các ngành: QT...
9552,2023-01-04,HSE Supervisor Upto 25 Millions,"2, 31","An toàn lao động, Môi trường - Xử lý chất thải",Quản lý cấp trung,16422000.0,25806000.0,2.0,NaN,Hà Nội,Khách hàng của Navigos Search là công ty sản x...,- Nam từ 28 - 38 tuổi - Tốt nghiệp các trường ...
9553,2023-01-04,Rất Gấp Kỹ Sư Cải Tiến,"41, 42","Sản phẩm công nghiệp, Sản xuất - Lắp ráp - Chế...",Chuyên viên - Nhân viên,11730000.0,18768000.0,3.0,NaN,Hà Nội,Khách hàng của Navigos Search là công ty R&D n...,"- Nam/nữ, dưới 27 tuổi - Tốt nghiệp Đại học tr..."


In [45]:
vnw_fi.to_csv("data/topcv/tidy_csv/vietnamworks_tidy.csv", index=False)

In [46]:
pd.read_csv('data/topcv/tidy_csv/vietnamworks_tidy.csv')

,date,title,industry,mapped_industry,level,min_salary,max_salary,min_year,max_year,city,description,requirements
0,2023-04-19,Senior Recruitment Consultant (Industrial Manu...,"5, 34, 52","Bán buôn - Bán lẻ - Quản lý cửa hàng, Nhân sự,...",Chuyên viên - Nhân viên,NaN,NaN,3.0,NaN,TP.HCM,"Navigos Search, a part of Navigos Group, is th...",- College Degree and above - At least 3 year...
1,2023-04-18,Nhân Viên Chất Lượng IQC ( Ưu Tiên Biết Tiếng ...,"39, 42, 60","Quản lý tiêu chuẩn và chất lượng, Sản xuất - L...",Chuyên viên - Nhân viên,NaN,NaN,2.0,NaN,"Bắc Ninh, Hà Nội",- Phụ trách quản lý chất lượng hàng kim loại. ...,- Có kinh nghiệm từ 2 năm trở lên làm chất lượ...
2,2023-04-21,Chuyên Viên Tư Vấn Bất Động Sản - Lương Cứng L...,"5, 7, 52","Bán buôn - Bán lẻ - Quản lý cửa hàng, Bất động...",Chuyên viên - Nhân viên,23460000.0,NaN,NaN,NaN,Hà Nội,Đất Xanh Miền Bắc là công ty bất động sản hàng...,"- Đam mê kinh doanh, muốn kiếm nhiều tiền và m..."
3,2023-04-29,Banking Product Development Officer,"30, 33, 50","Marketing, Ngân hàng, Tài chính - Đầu tư",Chuyên viên - Nhân viên,NaN,NaN,2.0,NaN,TP.HCM,Tasks: - Analyze competition of Banking produc...,Job requirements: - Male/ Female with Bachelor...
4,2023-04-21,Giám Sát Dịch Vụ Vệ Sinh Tòa Nhà Làm Việc Tại ...,"8, 25, 31","Chăm sóc khách hàng, Khách sạn - Nhà hàng - Du...",Chuyên viên - Nhân viên,8680200.0,11260800.0,1.0,NaN,Hà Nội,I. Thông tin chung: - Chức danh: ...,"– Nữ từ 24 – 40 tuổi, có sức khỏe tốt – Tốt ng..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9550,2023-01-04,"General Manager, Financial Services","10, 33, 50","Quản lý điều hành, Ngân hàng, Tài chính - Đầu tư",Quản lý cấp cao,NaN,NaN,NaN,NaN,"TP.HCM, Hà Nội",We are seeking an experienced and motivated Ge...,Minimum of 20 years of experience in sales and...
9551,2023-01-04,Giám Đốc Nhân Sự,"10, 18, 34","Quản lý điều hành, Hành chính - Thư ký, Nhân sự",Quản lý cấp cao,82110000.0,140760000.0,5.0,NaN,TP.HCM,"1/ Cấu trúc tổ chức - Xây dựng, thiết kế cấu ...",- Cử nhân/ Thạc sĩ trở lên trong các ngành: QT...
9552,2023-01-04,HSE Supervisor Upto 25 Millions,"2, 31","An toàn lao động, Môi trường - Xử lý chất thải",Quản lý cấp trung,16422000.0,25806000.0,2.0,NaN,Hà Nội,Khách hàng của Navigos Search là công ty sản x...,- Nam từ 28 - 38 tuổi - Tốt nghiệp các trường ...
9553,2023-01-04,Rất Gấp Kỹ Sư Cải Tiến,"41, 42","Sản phẩm công nghiệp, Sản xuất - Lắp ráp - Chế...",Chuyên viên - Nhân viên,11730000.0,18768000.0,3.0,NaN,Hà Nội,Khách hàng của Navigos Search là công ty R&D n...,"- Nam/nữ, dưới 27 tuổi - Tốt nghiệp Đại học tr..."
